## Import Python module

In [1]:
import pytesseract
import cv2
import re

In [2]:
import base64
from Crypto import Random
from Crypto.Cipher import AES
import secrets

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [4]:
import sqlite3
import string

In [5]:
import datetime
import time
import pandas as pd

In [6]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

#### 임시
****

In [7]:
phone = "galaxyS7"
p = "phone1"
img_num = 1

***

### 1. OCR
- OCR인식이 불확실한 점으로 인해 전처리 하는 방법에 의존하거나 제대로되 결과를 얻지 못하여 전처리 부분은 생략함

In [8]:
def upper_to_digit(upper):
    res = ''
    up = {'O':'0', 'G':'6', 'I':'1', 'S':'5'}
    for i in upper:
        if i in string.ascii_uppercase:
            res += up[i]
        else:
            res += i
    return res

def digit_to_upper(digit):
    res = ''
    dig = {'0':'O', '6':'G', '1':'I'}
    for i in digit:
        if i in string.digits:
            res += dig[i]
        else:
            res += i
    return res

In [9]:
# 전처리 함수 - 임계치
def prepare_threshold(url):
    img = cv2.imread(url)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gcopy = gray.copy()
    gcopy[gcopy >= 70] = 255
    gcopy[gcopy < 70] = 0
    
    #adap_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 25, 5)
    #dst = cv2.bitwise_not(adap_thresh)
    res = pytesseract.image_to_string(gcopy)
    txt = re.sub(pattern='[^\w\s]',repl='',string=res.replace(" ",""))

    return txt

In [10]:
# 전처리함수 - repadding
def prepare_repadding(url):
    image = Image.open(url)
#     area = (12,1,136,11) #워터마크 글자 크기에 따라 값이 다름
    cropped = image.crop(area)
    pad_size = (148, 13)
    
    background = (0,0,0)
    backg = Image.new('RGB', pad_size, background)
    backg.paste(cropped,area)
    
    save_url = 'extract/after/extract_{}_img{}.png'.format(p,img_num)
    backg.save(save_url)
    res = pytesseract.image_to_string(save_url)
    txt = re.sub(pattern='[^\w\s]',repl='',string=res.replace(" ",""))
    
    return txt
    

In [11]:
# upper/digit 변형
def transform(txt):
    idx = ''
    sites = txt[:3]
    date = txt[3:7]
    user_code = txt[7:13]
    user_num = txt[13:]

    idx = digit_to_upper(sites) + upper_to_digit(date) + digit_to_upper(user_code) + upper_to_digit(user_num)

    return idx

In [12]:
url = "extract/extract_{}_img{}.png".format(p,img_num)
res = pytesseract.image_to_string(url)
txt = re.sub(pattern='[^\w\s]',repl='',string=res.replace(" ",""))

if len(txt) == 17:
    idx = transform(txt)
else:
#     prepare_threshold(url)
#     prepare_repadding(url)
    print("OCR을 이용한 워터마크 추출 불가능")

### 2. user DB 연결

In [13]:
conn = sqlite3.connect("user_db/user.db")

#### * SQL문

In [14]:
select_key = "SELECT * from KEY"
select_webuser_pd = "SELECT * FROM webuser WHERE idx = '{}'"

### 2. KEY 불러오기

In [15]:
key_df = pd.read_sql_query(select_key ,conn)
key_dict = dict(zip(key_df.columns, key_df.iloc[0]))

### 3. 사용자 찾기

In [16]:
sus_df = pd.read_sql_query(select_webuser_pd.format(idx),conn)
enc_suspect = dict(zip(sus_df.columns, sus_df.iloc[0]))

### 4. 해독

In [17]:
class AES_cbcmode:
    
    def __init__(self,key):
        self.key = key
    
    def padding(self, raw_data):
        # 한국어의 경우 utf-8했을 때의 len이 달라지므로 encoding 과정 추가
        enc = raw_data.encode('utf-8')
        pad_num = AES.block_size - len(enc) % AES.block_size
        pad_chr = chr(pad_num)
        
        # encoding해서 최종 pad_data 반환
        return (raw_data + pad_num * pad_chr).encode('utf-8')
    
    def unpadding(self, pad_data):
        return pad_data[:-pad_data[-1]]
    
    def encrypt(self, raw_data):
        pad_data = self.padding(raw_data)
    
        # iv: initial vector
        # 두 번째 키와 같은 존재, 
        # 블록 단위로 암호/복호하면서 변경되고 그 값은 다음 블록 단위 암호/복호 시 사용
        iv = Random.new().read( AES.block_size )
        
        # CBC mode(Cipher Block Chaining)
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode(iv + cipher.encrypt(pad_data))
    
    def decrypt(self, enc_data):
        enc_decode = base64.b64decode(enc_data)
        iv = enc_decode[:AES.block_size]
        cipher = AES.new(self.key,AES.MODE_CBC, iv)
        return self.unpadding(cipher.decrypt(enc_decode[AES.block_size:]))

In [18]:
suspect = enc_suspect.copy()

des_ip = AES_cbcmode(key_dict['ipkey']).decrypt(enc_suspect['ip'])
suspect['ip'] = str(des_ip)[2:-1].replace("*", "")

des_device = AES_cbcmode(key_dict['devicekey']).decrypt(enc_suspect['device'])
suspect['device'] = str(des_device)[2:-1].replace("*", "")

des_pid = AES_cbcmode(key_dict['pidkey']).decrypt(enc_suspect['pid'])
suspect['pid'] = str(des_pid)[2:-1].replace("*", "")

In [19]:
print(suspect)

{'idx': 'DAU0614MGQOHR0000', 'year': 2020, 'month': 6, 'day': 14, 'time': '12:46:11', 'ip': '119.25.133.114', 'device': 'SM-G930S', 'pid': 'Q18MK5XI9M', 'url': 'http://webtoon.daum.net/webtoon/viewer/81878'}


#### DB종료

In [20]:
conn.close()